# Comikry — Colab Prototype
End-to-end pipeline validation on a single comic page.

**Run cells top-to-bottom.** Cell 2 clones the latest repo code automatically.

**API key setup (once):**
- **VS Code** — copy `.env.example` → `.env` in the repo root and fill in `OPENROUTER_API_KEY`
- **Colab browser** — left sidebar → 🔑 *Secrets* → *Add new secret* → name: `OPENROUTER_API_KEY`


In [1]:
import os
from pathlib import Path

REPO_URL  = 'https://github.com/LIdro/comikry.git'
REPO_DIR  = Path('/content/comikry')

if REPO_DIR.exists():
    # Already cloned — pull latest changes
    print('► Pulling latest changes …')
    os.system(f'git -C {REPO_DIR} pull --ff-only')
else:
    print('► Cloning comikry repo …')
    os.system(f'git clone --depth 1 {REPO_URL} {REPO_DIR}')

# Add repo root to the Python path so backend modules are importable if needed
import sys
if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

print(f'✅  Repo ready at {REPO_DIR}')


► Cloning comikry repo …
✅  Repo ready at /content/comikry
✅  Repo ready at /content/comikry


In [ ]:
# ── System packages ────────────────────────────────────────────────────────
!apt-get install -q -y ffmpeg poppler-utils

# ── Python packages ────────────────────────────────────────────────────────
!pip install -q pdf2image Pillow httpx python-ulid pydantic pydantic-settings python-dotenv

# ── Audiocraft ─────────────────────────────────────────────────────────────
# Install audiocraft's runtime deps first, then audiocraft itself with
# --no-build-isolation so setup.py can see the already-installed packages.
!pip install -q 'torch>=2.1' 'torchaudio>=2.1' einops flashy xformers encodec num2words sentencepiece huggingface_hub
!pip install -q --no-build-isolation --no-deps git+https://github.com/facebookresearch/audiocraft.git

# Verify in a subprocess (the current kernel's import cache is stale after pip install)
import subprocess, sys
_r = subprocess.run([sys.executable, '-c', 'import audiocraft; print(audiocraft.__name__)'], capture_output=True, text=True)
if _r.returncode == 0:
    print('✅  audiocraft installed successfully')
else:
    print('❌  audiocraft import failed:\n', _r.stderr)


Reading package lists...
Building dependency tree...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).

Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (277 kB/s)
Fetched 186 kB in 1s (

In [2]:
import os
from pathlib import Path

# ── Load API key ─────────────────────────────────────────────────────────────
# Supports two environments automatically:
#
#   VS Code  → reads from the root .env  (copy .env.example → .env and fill in key)
#   Colab    → reads from the 🔑 Secrets panel (left sidebar → Add new secret)
#
def _load_api_key() -> str:
    # Build candidate .env paths.
    # REPO_DIR is set by cell 2 (the git clone/pull cell) so it always points
    # to the repo root regardless of the kernel's working directory.
    candidates = [
        REPO_DIR / '.env',           # set by cell 2 — works in both VS Code and Colab
        Path('/content/comikry/.env'),  # Colab fallback if cell 2 was skipped
        Path(__file__).resolve().parents[1] / '.env'
            if '__file__' in dir() else None,
        Path('.env'),                # last-resort cwd fallback
    ]

    for env_file in candidates:
        if env_file is None or not env_file.exists():
            continue
        for line in env_file.read_text().splitlines():
            line = line.strip()
            if line.startswith('OPENROUTER_API_KEY=') and not line.startswith('#'):
                key = line.split('=', 1)[1].strip()
                if key and key != 'your_openrouter_api_key_here':
                    print(f'🔑  API key loaded from {env_file}')
                    return key

    # Fall back to Colab Secrets (browser Colab without a local .env)
    try:
        from google.colab import userdata
        key = userdata.get('OPENROUTER_API_KEY')
        if key:
            print('🔑  API key loaded from Colab Secrets')
            return key
    except Exception:
        pass

    # Already set in the process environment
    key = os.environ.get('OPENROUTER_API_KEY', '')
    if key:
        print('🔑  API key loaded from environment variable')
        return key

    raise EnvironmentError(
        'OPENROUTER_API_KEY not found.\n'
        '  VS Code : copy .env.example → .env in the repo root and fill in the key.\n'
        '  Colab   : left sidebar → 🔑 Secrets → Add new secret → OPENROUTER_API_KEY'
    )

os.environ['OPENROUTER_API_KEY'] = _load_api_key()

# ── Comic / run settings ─────────────────────────────────────────────────────
PDF_PATH   = 'comic.pdf'   # ← set the filename of the PDF you uploaded
PAGE_NUM   = 1             # page to process (1-based)
COMIC_ID   = 'colab_test'
STORAGE    = '/content/storage'
os.makedirs(STORAGE, exist_ok=True)

# ── Audiocraft device ────────────────────────────────────────────────────────
# 'auto' → CUDA if a GPU runtime is active, otherwise CPU (recommended)
# 'cuda' → force GPU  (Runtime → Change runtime type → T4 GPU)
# 'cpu'  → force CPU  (works on any runtime, ~3–5× slower for audiogen-small)
AUDIOCRAFT_DEVICE = 'auto'   # ← change to 'cuda' or 'cpu' to override


NameError: name 'REPO_DIR' is not defined

## Step 1 — Render PDF page to PNG

In [ ]:
from pdf2image import convert_from_path
from pathlib import Path

pages_dir = Path(STORAGE) / COMIC_ID / 'pages'
pages_dir.mkdir(parents=True, exist_ok=True)

pages = convert_from_path(PDF_PATH, dpi=150, fmt='png')
page_path = pages_dir / f'page_{PAGE_NUM:04d}.png'
pages[PAGE_NUM - 1].save(str(page_path), 'PNG')
print(f'Saved page image: {page_path}')

from IPython.display import display, Image as IPImage
display(IPImage(str(page_path), width=600))

## Step 2 — Detect panels with Gemini

In [ ]:
import base64, json, httpx

OPENROUTER_KEY  = os.environ['OPENROUTER_API_KEY']
VISION_MODEL    = 'google/gemini-2.5-flash-lite'
OR_BASE         = 'https://openrouter.ai/api/v1'
HEADERS         = {'Authorization': f'Bearer {OPENROUTER_KEY}', 'Content-Type': 'application/json'}

def encode_image(path):
    return base64.b64encode(open(path, 'rb').read()).decode()

def chat(model, messages):
    r = httpx.post(f'{OR_BASE}/chat/completions',
                   headers=HEADERS, json={'model': model, 'messages': messages}, timeout=120)
    r.raise_for_status()
    return r.json()['choices'][0]['message']['content'].strip()

PANEL_PROMPT = '''
You are a comic panel analyser. Return ONLY a JSON array of panels in reading order.
Each element: {"order": int, "x": int, "y": int, "w": int, "h": int}
'''

b64 = encode_image(str(page_path))
raw = chat(VISION_MODEL, [
    {'role': 'system', 'content': PANEL_PROMPT},
    {'role': 'user', 'content': [
        {'type': 'image_url', 'image_url': {'url': f'data:image/png;base64,{b64}'}},
        {'type': 'text', 'text': 'Detect all panels.'}
    ]}
])

if raw.startswith('```'): raw = raw.split('```')[1].lstrip('json').strip()
panels_raw = json.loads(raw)
print(f'Found {len(panels_raw)} panels:', json.dumps(panels_raw, indent=2))

## Step 3 — Crop panels and run bubble OCR

In [ ]:
from PIL import Image as PILImage

panels_dir = Path(STORAGE) / COMIC_ID / 'panels'
panels_dir.mkdir(exist_ok=True)

BUBBLE_PROMPT = '''
You are a comic OCR specialist. For each bubble return ONLY a JSON array.
Each element: {"order": int, "type": str, "x": int, "y": int, "w": int, "h": int, "text": str, "confidence": float}
Types: speech, thought, narration, sfx.
'''

all_bubbles = []
page_img = PILImage.open(str(page_path))

for p in sorted(panels_raw, key=lambda d: d['order']):
    crop = page_img.crop((p['x'], p['y'], p['x']+p['w'], p['y']+p['h']))
    panel_file = panels_dir / f"panel_{p['order']:03d}.png"
    crop.save(str(panel_file), 'PNG')

    b64p = encode_image(str(panel_file))
    raw = chat(VISION_MODEL, [
        {'role': 'system', 'content': BUBBLE_PROMPT},
        {'role': 'user', 'content': [
            {'type': 'image_url', 'image_url': {'url': f'data:image/png;base64,{b64p}'}},
            {'type': 'text', 'text': 'Extract all bubbles and their text.'}
        ]}
    ])
    if raw.startswith('```'): raw = raw.split('```')[1].lstrip('json').strip()
    bubbles = json.loads(raw)
    for b in bubbles:
        b['panel_order'] = p['order']
    all_bubbles.extend(bubbles)
    print(f"Panel {p['order']}: {len(bubbles)} bubbles")

print(json.dumps(all_bubbles, indent=2))

## Step 4 — Speaker attribution + voice assignment

In [ ]:
ATTR_PROMPT = '''
You are a comic character recognition agent. Attribute each bubble to a speaker.
Return ONLY: {"attributions": [{"bubble_id": str, "speaker_id": str, "label": str}],
              "new_speakers": [{"speaker_id": str, "label": str, "gender": str, "age_group": str}]}
Use \"narrator\" for narration boxes. Reuse known speaker IDs when the same character reappears.
'''

VOICE_MAP = {
    ('male','child'): 'verse', ('male','teen'): 'verse', ('male','adult'): 'echo', ('male','elder'): 'onyx',
    ('female','child'): 'coral', ('female','teen'): 'coral', ('female','adult'): 'nova', ('female','elder'): 'shimmer',
    ('unknown','adult'): 'alloy',
}

known_speakers = []

for p in sorted(panels_raw, key=lambda d: d['order']):
    panel_file = panels_dir / f"panel_{p['order']:03d}.png"
    panel_bubbles = [b for b in all_bubbles if b['panel_order'] == p['order']]
    bubble_list = [{'bubble_id': f"p{p['order']}_b{b['order']}", 'text': b['text'], 'type': b['type']} for b in panel_bubbles]

    b64p = encode_image(str(panel_file))
    raw = chat(VISION_MODEL, [
        {'role': 'system', 'content': ATTR_PROMPT},
        {'role': 'user', 'content': [
            {'type': 'image_url', 'image_url': {'url': f'data:image/png;base64,{b64p}'}},
            {'type': 'text', 'text': json.dumps({'known_speakers': [{'speaker_id': s['speaker_id'], 'label': s['label']} for s in known_speakers], 'bubbles': bubble_list})}
        ]}
    ])
    if raw.startswith('```'): raw = raw.split('```')[1].lstrip('json').strip()
    attr_data = json.loads(raw)

    for attr in attr_data.get('attributions', []):
        for b in all_bubbles:
            if f"p{p['order']}_b{b['order']}" == attr['bubble_id']:
                b['speaker_id'] = attr['speaker_id']

    existing_ids = {s['speaker_id'] for s in known_speakers}
    for ns in attr_data.get('new_speakers', []):
        if ns['speaker_id'] not in existing_ids:
            ns['voice_id'] = VOICE_MAP.get((ns.get('gender','unknown'), ns.get('age_group','adult')), 'alloy')
            known_speakers.append(ns)

print('Speakers:', json.dumps(known_speakers, indent=2))
print('Attributed bubbles:', json.dumps(all_bubbles, indent=2))

## Step 5 — Emotion tagging + TTS generation

In [ ]:
TTS_MODEL = 'openai/gpt-audio-mini'
voice_dir = Path(STORAGE) / COMIC_ID / 'audio' / 'voice'
voice_dir.mkdir(parents=True, exist_ok=True)

EMOTION_PROMPT = '''
Infer emotion per bubble. Return ONLY a JSON array:
[{"bubble_id": str, "emotion": str}]
Emotions: neutral, happy, sad, angry, excited, scared, surprised, sarcastic, whispering, shouting.
'''

speaker_map = {s['speaker_id']: s['label'] for s in known_speakers}
bubble_list_for_emotion = [{'bubble_id': f"p{b['panel_order']}_b{b['order']}", 'text': b['text'], 'speaker': speaker_map.get(b.get('speaker_id',''), 'unknown')} for b in all_bubbles if b.get('text','').strip()]

raw = chat(VISION_MODEL, [
    {'role': 'system', 'content': EMOTION_PROMPT},
    {'role': 'user', 'content': json.dumps(bubble_list_for_emotion)}
])
if raw.startswith('```'): raw = raw.split('```')[1].lstrip('json').strip()
emotion_data = json.loads(raw)
emotion_map = {e['bubble_id']: e['emotion'] for e in emotion_data}

for b in all_bubbles:
    bid = f"p{b['panel_order']}_b{b['order']}"
    b['emotion'] = emotion_map.get(bid, 'neutral')

# Generate TTS
voice_lookup = {s['speaker_id']: s['voice_id'] for s in known_speakers}

for b in all_bubbles:
    if not b.get('text', '').strip() or b.get('type') == 'sfx': continue
    voice_id = voice_lookup.get(b.get('speaker_id', ''), 'alloy')
    out_path = voice_dir / f"p{b['panel_order']}_b{b['order']}.mp3"
    r = httpx.post(f'{OR_BASE}/audio/speech', headers=HEADERS, timeout=120,
                   json={'model': TTS_MODEL, 'input': b['text'], 'voice': voice_id,
                         'instructions': f"Speak with a {b['emotion']} tone.", 'response_format': 'mp3'})
    r.raise_for_status()
    out_path.write_bytes(r.content)
    b['audio_path'] = str(out_path)
    print(f"  [{b['emotion']}] {b['text'][:50]!r} → {out_path.name}")

## Step 6 — SFX generation (Audiocraft AudioGen)

Uses `facebook/audiogen-small` (~300 MB) — small enough to run comfortably on **both CPU and GPU**.

| Mode | When to use | Speed |
|---|---|---|
| **GPU** (`cuda`) | Colab T4 runtime — *Runtime → Change runtime type → T4 GPU* | ~4 s per panel |
| **CPU** (`cpu`) | Free Colab CPU runtime or when no GPU is available | ~15–25 s per panel |

Set `AUDIOCRAFT_DEVICE` in the config cell above to `'auto'` (default), `'cuda'`, or `'cpu'`.

Outputs are converted from WAV → 128 kbps MP3 via ffmpeg then the WAV is deleted to reclaim disk space.  
Model weights are cached under `/content/audiocraft_cache` (outside the home partition) to avoid quota issues.  
Weights are re-downloaded each session (~300 MB) unless you persist that folder to Google Drive.


In [ ]:
import os, json, subprocess, torch
from pathlib import Path
from audiocraft.models import AudioGen
from audiocraft.data.audio import audio_write

# ── Resolve device from the config cell setting ────────────────────────────
if AUDIOCRAFT_DEVICE == 'auto':
    _device = 'cuda' if torch.cuda.is_available() else 'cpu'
elif AUDIOCRAFT_DEVICE == 'cuda':
    if not torch.cuda.is_available():
        print('⚠️  CUDA requested but no GPU found — falling back to CPU.')
        print('    To use a GPU: Runtime → Change runtime type → T4 GPU')
        _device = 'cpu'
    else:
        _device = 'cuda'
else:
    _device = 'cpu'

print(f'Audiocraft device: {_device}')

# ── Redirect model weights away from the home-partition quota ──────────────
os.environ['AUDIOCRAFT_CACHE_DIR'] = '/content/audiocraft_cache'
os.makedirs('/content/audiocraft_cache', exist_ok=True)

# ── Load model (audiogen-small ≈ 300 MB — fits on T4 and runs fine on CPU) ─
print('Loading facebook/audiogen-small …')
sfx_model = AudioGen.get_pretrained('facebook/audiogen-small', device=_device)
sfx_model.set_generation_params(duration=4)   # 4 s per panel: sufficient & saves VRAM/disk
print('Model ready.')

# ── Build one SFX prompt per panel via Gemini ──────────────────────────────
SFX_PROMPT = '''
You are a comic sound-design director. For each panel image produce a short, vivid
audio description suitable as a text-to-audio prompt (≤ 12 words).
Return ONLY a JSON object: {"panel_<order>": "<prompt>", ...}
Focus on ambient mood + key action sounds. Avoid music references.
'''

sfx_prompts: dict[str, str] = {}
for p in sorted(panels_raw, key=lambda d: d['order']):
    panel_file = panels_dir / f"panel_{p['order']:03d}.png"
    b64p = encode_image(str(panel_file))
    raw = chat(VISION_MODEL, [
        {'role': 'system', 'content': SFX_PROMPT},
        {'role': 'user', 'content': [
            {'type': 'image_url', 'image_url': {'url': f'data:image/png;base64,{b64p}'}},
            {'type': 'text', 'text': 'Describe the soundscape for this panel.'}
        ]}
    ])
    if raw.startswith('```'):
        raw = raw.split('```')[1].lstrip('json').strip()
    try:
        sfx_prompts.update(json.loads(raw))
    except json.JSONDecodeError:
        sfx_prompts[f'panel_{p["order"]}'] = 'soft ambient background, comic book atmosphere'

print('SFX prompts:', json.dumps(sfx_prompts, indent=2))

# ── Generate audio + convert WAV → MP3 (saves ≈ 60 % disk) ────────────────
sfx_dir = Path(STORAGE) / COMIC_ID / 'audio' / 'sfx'
sfx_dir.mkdir(parents=True, exist_ok=True)

for p in sorted(panels_raw, key=lambda d: d['order']):
    key      = f'panel_{p["order"]}'
    prompt   = sfx_prompts.get(key, 'soft ambient background, comic book atmosphere')
    stem     = str(sfx_dir / f'panel_{p["order"]:03d}')
    wav_path = Path(stem + '.wav')
    mp3_path = Path(stem + '.mp3')

    with torch.inference_mode():
        wav = sfx_model.generate([prompt])   # shape: (1, channels, samples)

    # audio_write appends '.wav' automatically
    audio_write(
        stem, wav[0].cpu(), sfx_model.sample_rate,
        strategy='loudness', loudness_compressor=True
    )

    # Convert to MP3 and remove the WAV to save disk
    result = subprocess.run(
        ['ffmpeg', '-y', '-i', str(wav_path), '-b:a', '128k', str(mp3_path)],
        capture_output=True
    )
    if result.returncode == 0 and wav_path.exists():
        wav_path.unlink()
        final_path = str(mp3_path)
    else:
        final_path = str(wav_path)   # keep WAV if ffmpeg failed

    p['sfx_path'] = final_path
    print(f'Panel {p["order"]:2d}: {Path(final_path).name}  |  prompt: {prompt!r}')

print('\nSFX generation complete.')


## Step 7 — Playback test


In [ ]:
from IPython.display import Audio as IPAudio, display, Image as IPImage

for p in sorted(panels_raw, key=lambda d: d['order']):
    panel_file = panels_dir / f"panel_{p['order']:03d}.png"
    print(f'\n=== Panel {p["order"]} ===')
    display(IPImage(str(panel_file), width=500))

    # SFX track for this panel
    if p.get('sfx_path') and Path(p['sfx_path']).exists():
        print(f'  🔊 SFX: {Path(p["sfx_path"]).name}')
        display(IPAudio(p['sfx_path']))

    # Dialogue / narration bubbles
    panel_bubbles = sorted(
        [b for b in all_bubbles if b['panel_order'] == p['order']],
        key=lambda x: x['order']
    )
    for b in panel_bubbles:
        if b.get('audio_path') and Path(b['audio_path']).exists():
            print(f'  🗨  [{b.get("speaker_id","?")}] [{b["emotion"]}] {b["text"]}')
            display(IPAudio(b['audio_path']))
